In [ ]:
'''
SolO data search and download code through JSOC  
'''
%matplotlib inline

import astropy.units as us
import astropy.io.fits
import astropy.wcs
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.nddata import Cutout2D
from astropy.visualization import ImageNormalize, SqrtStretch, PowerStretch, AsinhStretch
from astropy.visualization import quantity_support
quantity_support()

from sunpy.net import Fido, attrs
import sunpy.map
import sunpy_soar
from sunpy import timeseries as ts
import sunpy.net.attrs as a
from sunpy.coordinates import get_earth, get_horizons_coord, propagate_with_solar_surface
import sunpy.visualization.drawing
from sunkit_instruments import iris
import sunraster.instr.spice

import dkist
import dkist.net
import pathlib
import warnings
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

import os
import sunpy_soar
import drms
import pandas as pd
import astropy.time
from sunpy.net import Fido, attrs as a
from sunpy.net import jsoc

def display_sdo_solo_images(download_dir, vmax_factor):
    """
    读取并显示下载目录中的图像文件，设置 vmax 为每个图像的均值的 10 倍，
    每张图像单独显示，标题为文件名。
    
    参数:
        download_dir (str): 包含要显示的图像文件的目录路径。
    """
    # 检查目录是否存在
    if not os.path.exists(download_dir):
        print(f"Directory does not exist: {download_dir}")
        return

    # 使用 sunpy 读取 FITS 文件
    files_euv = [os.path.join(download_dir, file) for file in os.listdir(download_dir) if file.endswith('.fits')]

    # 使用 SunPy Map 读取并处理每个 FITS 文件
    sequence = sunpy.map.Map(files_euv, sequence=True)

    # 设置每个图像的归一化显示
    for i, each_map in enumerate(sequence):
        # 计算每个图像的均值
        mean_value = np.mean(each_map.data)
        
        # 设置 vmax 为均值的 10 倍
        vmax_value = mean_value * vmax_factor
        
        # 设置图像的归一化显示
        each_map.plot_settings['norm'] = ImageNormalize(vmin=0, vmax=vmax_value, stretch=SqrtStretch())

        # 获取文件名作为标题
        filename = os.path.basename(files_euv[i])  # 获取文件名（不包括路径）
        
        # 创建一个新的 figure 并显示当前图像
        plt.figure(i)
        each_map.plot()
        plt.title(f"File: {filename}")  # 使用文件名作为标题
        plt.show()

#————————————————————————————————————————————————————————————————————————————————

# if search is needed only, make this 'False'
# if search is done and you need to download, make this 'True'
Whether_download = False   
# if there are many download files, make this 'False'
Whether_show_all_download_img = False

# parameters of the data needed to search or/and download
start_time = '2023-04-23 11:55:00'
end_time = '2023-04-23 12:15:00'
#start_time = '2023-10-02 12:50:00'
#end_time = '2023-10-02 12:59:00'
Instrument_Keyword = 'EUI'
Product_Keyword = 'EUI-HRIEUV174-IMAGE'

#Instrument_Keyword = 'EUI'
#Product_Keyword = 'EUI-FSI174-IMAGE'

Download_path = '/data2/pqf_SDO_SolO/data_SolO/test_downloads/'
vmax_factor = 50
plt.rcParams["figure.figsize"] = (12, 12)  # make plots GREAT again

#————————————————————————————————————————————————————————————————————————————————

instrument = a.Instrument(Instrument_Keyword)
time = a.Time(start_time, end_time)
level = a.Level(2)
product = a.soar.Product(Product_Keyword)

result = Fido.search(instrument & time & level & product)

print(result)

if Whether_download :
    download_dir = Download_path + f'{Product_Keyword}_{start_time}_to_{end_time}/'

    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
        print(f"Created download path: {download_dir}")
    else:
        print(f"Download path already exists: {download_dir}")

    files_euv = Fido.fetch(result, path = download_dir)
    print(files_euv)

    if Whether_show_all_download_img:
        display_sdo_solo_images(download_dir, vmax_factor)



# don't forget to fill in the parameters of the data needed


Results from 1 Provider:

180 Results from the SOARClient:

Instrument     Data product    ...               SOOP Name               
                               ...                                       
---------- ------------------- ... --------------------------------------
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD_Slow-Wind-Connection
       EUI eui-hrieuv174-image ... L_SMALL_HRES_HCAD

Files Downloaded:   0%|          | 0/180 [00:00<?, ?file/s]

5/0 files failed to download. Please check `.errors` for details
['/data2/pqf_SDO_SolO/data_SolO/test_downloads/EUI-HRIEUV174-IMAGE_2023-04-23 11:55:00_to_2023-04-23 12:15:00/solo_L2_eui-hrieuv174-image_20230423T120000194_V01.fits', '/data2/pqf_SDO_SolO/data_SolO/test_downloads/EUI-HRIEUV174-IMAGE_2023-04-23 11:55:00_to_2023-04-23 12:15:00/solo_L2_eui-hrieuv174-image_20230423T120005191_V01.fits', '/data2/pqf_SDO_SolO/data_SolO/test_downloads/EUI-HRIEUV174-IMAGE_2023-04-23 11:55:00_to_2023-04-23 12:15:00/solo_L2_eui-hrieuv174-image_20230423T120010191_V01.fits', '/data2/pqf_SDO_SolO/data_SolO/test_downloads/EUI-HRIEUV174-IMAGE_2023-04-23 11:55:00_to_2023-04-23 12:15:00/solo_L2_eui-hrieuv174-image_20230423T120015191_V01.fits', '/data2/pqf_SDO_SolO/data_SolO/test_downloads/EUI-HRIEUV174-IMAGE_2023-04-23 11:55:00_to_2023-04-23 12:15:00/solo_L2_eui-hrieuv174-image_20230423T120020196_V01.fits']
Errors:
(<parfive.results.Error object at 0x767f27d04220>
http://soar.esac.esa.int/soar-sl-tap/data

In [22]:
from sunpy.net import Fido, attrs as a

# Example search query
instrument = a.Instrument('SPICE')
time = a.Time('2023-12-31 23:30:00', '2023-12-31 23:59:59')
result = Fido.search(instrument & time)

# Check if there are results
if result:
    # Access the first result and print it
    #result_table = result[0]  # This is already a table-like object
    #print(result_table)
    print(result)
else:
    print("No results found")


Results from 1 Provider:

0 Results from the SOARClient:

Instrument Data product Level Start time End time Filesize SOOP Name
                                                   Mbyte            
---------- ------------ ----- ---------- -------- -------- ---------


